In [1]:
!pip install transformers
!pip install langchain
!pip install datasets
!pip install torch torchvision torchaudio
from datasets import load_dataset


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 43.1 MB/s eta 0:00:00
  Attempting uninsta

In [ ]:
data_files = {
    "train": "CNNTrain.csv",
    "test": "CNNTest.csv",
    "validation": "CNNValidation.csv"
}

dataset = load_dataset('csv', data_files=data_files)

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")

def preprocess_function(examples):
    inputs = examples['article']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['highlights'], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig

# Load the model
model = transformers.AutoModelForCausalLM.from_pretrained(
  'tiiuae/falcon-7b-instruct',
  trust_remote_code=True
)

# Apply LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Reduced batch size
    per_device_eval_batch_size=1,   # Reduced batch size
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=10_000,
    fp16=True,  # Enable mixed precision training
    gradient_accumulation_steps=8,  # Accumulate gradients over 8 steps
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"]
)

# Train the model
trainer.train()


In [ ]:
from transformers import pipeline
from langchain import HuggingFacePipeline, PromptTemplate,  LLMChain


# Set up the pipeline for text generation
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=100, do_sample=True, use_cache=True, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)

# Wrap the pipeline in HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})

# Define the prompt template
template = """
Write a concise summary of the following text delimited by triple backquotes.
```{text}```
SUMMARY:
"""

prompt = PromptTemplate(template=template, input_variables=["text"])

# Create LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
df = pd.DataFrame(dataset['test'])[['article', 'highlights','id']]

In [ ]:
df.head()

,article,highlights,id
0,CNNThe Palestinian Authority officially became...,Membership gives the ICC jurisdiction over all...,f001ec5c4704938247d27a44948eebb37ae98d01
1,CNNNever mind cats having nine lives. A stray ...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef
2,CNNIf youve been following the news lately the...,Mohammad Javad Zarif has spent more time with ...,4495ba8f3a340d97a9df1476f8a35502bcce1f69
3,CNNFive Americans who were monitored for three...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca
4,CNNA Duke student has admitted to hanging a no...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f


In [ ]:
df.drop(columns=['id'], inplace=True)
df.head()

,article,highlights
0,CNNThe Palestinian Authority officially became...,Membership gives the ICC jurisdiction over all...
1,CNNNever mind cats having nine lives. A stray ...,"Theia, a bully breed mix, was apparently hit b..."
2,CNNIf youve been following the news lately the...,Mohammad Javad Zarif has spent more time with ...
3,CNNFive Americans who were monitored for three...,17 Americans were exposed to the Ebola virus w...
4,CNNA Duke student has admitted to hanging a no...,Student is no longer on Duke University campus...


In [ ]:
df['model_generated'] = ""

# Define a function to generate summaries and populate the 'model_generated' column
def generate_and_store_summary(row):
    article_text = row['article']
    summary = llm_chain.run(article_text)
    return summary

In [ ]:
df['model_generated'] = df.apply(generate_and_store_summary, axis=1)

In [ ]:
# Apply the logic to all rows of the 'model_generated' column
df['model_generated'] = df['model_generated'].apply(lambda text: text.split('SUMMARY:')[1].strip() if 'SUMMARY:' in text else text)

In [5]:
print(df[['article', 'model_generated']].head(10))

                                             article  \
0  CNNThe Palestinian Authority officially became...   
1  CNNNever mind cats having nine lives. A stray ...   
2  CNNIf youve been following the news lately the...   
3  CNNFive Americans who were monitored for three...   
4  CNNA Duke student has admitted to hanging a no...   
5  CNNHes a blue chip college basketball recruit....   
6  CNNGovernments around the world are using the ...   
7  CNNAndrew Getty one of the heirs to billions o...   
8  CNNFilipinos are being warned to be on guard f...   
9  CNNFor the first time in eight years a TV lege...   

                                     model_generated  
0  International Criminal Court has now 123 membe...  
1  The dog, Theia, who was thought dead after bei...  
2  Mr. Ambassador: A memoir of one Iranian in a p...  
3  - 4+ Americans who had been quarantined in Oma...  
4  - College student with noose “hangs” it near c...  
5  In a high school gym, a special needs girl nam... 

In [9]:
df['article'][8]

'CNNFilipinos are being warned to be on guard for flash floods and landslides as tropical storm Maysak approached the Asian island nation Saturday. Just a few days ago Maysak gained super typhoon status thanks to its sustained 150 mph winds. It has since lost a lot of steam as it has spun west in the Pacific Ocean. Its now classified as a tropical storm according to the Philippine national weather service which calls it a different name Chedeng. It boasts steady winds of more than 70 mph 115 kph and gusts up to 90 mph as of 5 p.m. 5 a.m. ET Saturday. Still that doesnt mean Maysak wont pack a wallop. Authorities took preemptive steps to keep people safe such as barring outdoor activities like swimming surfing diving and boating in some locales as well as a number of precautionary evacuations. Gabriel Llave a disaster official told PNA that tourists who arrive Saturday in and around the coastal town of Aurora will not be accepted by the owners of hotels resorts inns and the like ... and 

In [10]:
df['model_generated'][8]

'A super typhoon moving west brought serious concern to Filipinos Sunday morning. Authorities took necessary precautions with evacuations, no outdoor activities, and a specific hotel policy.'

In [14]:
!pip install rouge
!pip install nltk
!pip install bert_score

from rouge import Rouge

# Initialize the ROUGE evaluator
rouge = Rouge()

sampled_df = df.dropna(subset=['model_generated', 'highlights'])

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

# Calculate ROUGE scores for the selected samples
rouge_scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

from nltk.translate.bleu_score import corpus_bleu


# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

# Calculate BLEU score for the selected samples
bleu_score = corpus_bleu(reference_summaries, generated_summaries)

from bert_score import score

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

# Calculate BERT Score
P, R, F1 = score(generated_summaries, reference_summaries, lang="en", verbose=True)


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.72 seconds, 13.85 sentences/sec


In [15]:
# Print BERT Score
print("ROUGE Scores:", rouge_scores)
print("BLEU Score for Summaries:", bleu_score)
print("BERT Precision:", P.mean().item())
print("BERT Recall:", R.mean().item())
print("BERT F1 Score:", F1.mean().item())

ROUGE Scores: {'rouge-1': {'r': 0.2748651562180974, 'p': 0.17366179037714172, 'f': 0.2068370010676258}, 'rouge-2': {'r': 0.04749552615406274, 'p': 0.025647032783723333, 'f': 0.03211648712033251}, 'rouge-l': {'r': 0.23505825688178633, 'p': 0.14947103638238043, 'f': 0.17777519303265638}}
BLEU Score for Summaries: 9.435647963507199e-232
BERT Precision: 0.8139855265617371
BERT Recall: 0.8588894009590149
BERT F1 Score: 0.835396945476532
